In [ ]:
#| default_exp prune.prune_callback

In [ ]:
#| include: false
from nbdev.showdoc import *

In [ ]:
#| export
from fastai.vision.all import *
from fastai.callback.all import *
from fasterai.prune.pruner import *
from fasterai.core.criteria import *
from fasterai.core.schedule import *

import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
#| export
class PruneCallback(Callback):
    def __init__(self, pruning_ratio, schedule, context, criteria, *args, **kwargs):
        store_attr()
        self.sparsity_levels = []
        self.extra_args = args
        self.extra_kwargs = kwargs

    def before_fit(self):
        n_batches_per_epoch = len(self.learn.dls.train)
        total_training_steps = n_batches_per_epoch * self.learn.n_epoch
        self.pruning_ratio = self.pruning_ratio/100 if self.pruning_ratio>1 else self.pruning_ratio

        self.example_inputs, _ = self.learn.dls.one_batch()
        self.sparsity_levels = self.schedule._scheduler(self.pruning_ratio, total_training_steps)
        
        self.pruner = Pruner(
        self.learn.model,
        criteria=self.criteria,
        pruning_ratio=self.pruning_ratio, 
        context=self.context,
        iterative_steps= total_training_steps, 
        schedule=self.schedule._scheduler,
        *self.extra_args, 
        **self.extra_kwargs
        )
        
    def before_step(self):
        if self.training: 
            self.pruner.prune_model()

    def after_epoch(self):
        completed_steps = (self.epoch + 1) * len(self.learn.dls.train)
        current_sparsity = self.sparsity_levels[completed_steps - 1]
        print(f'Sparsity at the end of epoch {self.epoch}: {current_sparsity*100:.2f}%')

In [ ]:
show_doc(PruneCallback)

---

[source](https://github.com/FasterAI-Labs/fasterai/tree/master/blob/master/fasterai/prune/prune_callback.py#L18){target="_blank" style="float:right; font-size:smaller"}

### PruneCallback

>      PruneCallback (pruning_ratio, schedule, global_pruning, criteria, *args,
>                     **kwargs)

*Basic class handling tweaks of the training loop by changing a `Learner` in various events*